# Explore here

It's recommended to use this notebook for exploration purposes.

In [25]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import io

In [26]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",  # Pretend to be a browser
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8"
}

In [27]:
# Fetch the webpage

url = 'https://en.wikipedia.org/wiki/List_of_Spotify_streaming_records'
response = requests.get(url,headers=headers) 

In [28]:
print("RESPONSE STATUS IS: ", response.status_code)

RESPONSE STATUS IS:  200


In [29]:
# Extract tables with pandas
html = io.StringIO(response.text)  # Convert the HTML to a text file

# read_html() returns a list of DataFrames
tables = pd.read_html(html) # Scans for <table> tags and converts it into pandas DataFrame. Built to only parse for <table> default behavior
print(f"{len(tables)} tables were found.") # Checks the length of "tables" variable

27 tables were found.


In [30]:
df = tables[0]
df

,Rank,Song,Artist(s),Streams (billions),Release date,Ref.
0,1,"""Blinding Lights""",The Weeknd,5.030,29 November 2019,[1]
1,2,"""Shape of You""",Ed Sheeran,4.555,6 January 2017,[2]
2,3,"""Starboy""",The Weeknd and Daft Punk,4.096,21 September 2016,[3]
3,4,"""Someone You Loved""",Lewis Capaldi,4.055,8 November 2018,[4]
4,5,"""As It Was""",Harry Styles,4.041,1 April 2022,[5]
...,...,...,...,...,...,...
96,97,"""Levitating""",Dua Lipa and DaBaby,2.413,1 October 2020,[96]
97,98,"""Sweet Child O' Mine""",Guns N' Roses,2.409,3 June 1988,[97]
98,99,"""New Rules""",Dua Lipa,2.401,2 June 2017,NaN
99,100,"""Someone Like You""",Adele,2.401,24 January 2011,NaN
